In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#scraping URL where release is uploaded weekly
my_url = "https://www.dol.gov/newsroom/releases/"
raw_html = requests.get(my_url).content

In [3]:
soup_doc = BeautifulSoup(raw_html, "html.parser")
print(soup_doc.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <link href="https://www.dol.gov/newsroom/releases" rel="canonical"/>
  <meta content="DOL" property="og:site_name"/>
  <meta content="http://www.dol.gov/newsroom/releases" property="og:url"/>
  <meta content="News Releases" property="og:title"/>
  <meta content="http://www.dol.gov/sites/dolgov/files/OPA/twitter-cards/HomepageTwittercardOmbre.png" property="og:image"/>
  <meta content="http://www.dol.gov/sites/dolgov/files/OPA/twitter-cards/HomepageTwittercardOmbre.png" property="og:image:url"/>
  <meta content="https://www.dol.gov/sites/dolgo

In [4]:
# Find the element containing the specified text
text = soup_doc.find('span', string="Unemployment Insurance Weekly Claims Report")
link = text.find_parent('a')['href'] 
print(link)

    /newsroom/releases/eta/eta20231207



In [5]:
base_url = "https://www.dol.gov"
pdf_url = base_url + link.strip()
print (pdf_url)

https://www.dol.gov/newsroom/releases/eta/eta20231207


In [6]:
# Scraping PDFs
import camelot

In [7]:
tables = camelot.read_pdf(pdf_url, flavor='stream', pages='1-end')
len(tables)

/Users/radhikarukmangadhan/.pyenv/versions/3.11.6/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


10

In [8]:
# Find table with relevant info
df = tables[5].df
df

,0,1,2,3,4,5,6
0,,,Advance State Claims - Not Seasonally Adjusted,,,,
1,,,Initial Claims Filed During Week Ended December 2,,,Insured Unemployment For Week Ended November 25,
2,STATE,Advance,Prior Wk,Change,Advance,Prior Wk,Change
3,Alabama,"2,799","1,480","1,319","8,788","8,329",459
4,Alaska,"1,121",734,387,"6,363","6,061",302
5,Arizona,"3,143","2,464",679,"20,765","21,276",-511
6,Arkansas,"2,200","1,582",618,"9,686","8,706",980
7,California,"51,535","37,478","14,057","415,887","323,487","92,400"
8,Colorado,"2,803","2,111",692,"25,727","24,565","1,162"
9,Connecticut,"3,651","3,213",438,"24,070","20,709","3,361"


In [10]:
# Dropping unwanted rows and columns
df = df.drop(columns=[1,2,4, 5, 6])
df = df.drop([0, 1, 2])
df

,0,3
3,Alabama,"1,319"
4,Alaska,387
5,Arizona,679
6,Arkansas,618
7,California,"14,057"
8,Colorado,692
9,Connecticut,438
10,Delaware,-399
11,District of Columbia,189
12,Florida,"1,832"


In [13]:
# Naming columns
df.columns=["State","Change"]
df = df.replace({',': '','\*': ''}, regex=True)
df

,State,Change
3,Alabama,1319
4,Alaska,387
5,Arizona,679
6,Arkansas,618
7,California,14057
8,Colorado,692
9,Connecticut,438
10,Delaware,-399
11,District of Columbia,189
12,Florida,1832


In [14]:
# Downloading CSV in append mode
df.to_csv("statewise_claims.csv", index = False)